# **Importing libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir dataset

In [ ]:
!cp /content/drive/MyDrive/datajpg/train.csv /content/dataset

In [ ]:
!pip install imbalanced-learn

In [ ]:
!pip install imgaug

In [ ]:
from google.colab import files
import pandas as pd
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import imblearn
from imblearn.over_sampling import RandomOverSampler
from skimage.filters import threshold_otsu
import imgaug.augmenters as iaa

# **Downloading dataset from website**

In [ ]:
%cd /content/dataset

In [ ]:
!wget "https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_JPEG.zip"

# **Unzipping dataset**

In [ ]:
#!mkdir dataset

In [ ]:
#%cd dataset

In [ ]:
#!mv /content/ISIC_2020_Training_JPEG.zip /content/dataset/.

In [ ]:
!unzip "/content/dataset/ISIC_2020_Training_JPEG.zip"

In [ ]:
#!unzip "/content/dataset/train.csv.zip"

In [ ]:
main_path = "/content/dataset/train"
len(os.listdir(main_path))

In [ ]:
df = pd.read_csv("/content/dataset/train.csv")

In [ ]:
df.info()

In [ ]:
df['image_name_'] = df['image_name'].apply(lambda x: f"{main_path}/{x}.jpg")

In [ ]:
df

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import matplotlib.pyplot as plt


In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 256)))
    assert model.output_shape == (None, 8, 8, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 8, 8, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 64, 64, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 128, 128, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 256, 256, 3)

    return model


In [ ]:
generator = make_generator_model()


In [ ]:
generator.summary()

In [ ]:
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

In [ ]:
np.max(generated_image)

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[256, 256, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


In [ ]:
discriminator = make_discriminator_model()


In [ ]:
discriminator.summary()

In [ ]:
len(discriminator.layers)

In [ ]:
decision = discriminator(generated_image)
print(decision)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

In [ ]:
def save_models(generator, discriminator, epoch):
  """ Save models at specific point in time. """
  tf.keras.models.save_model(
    generator,
    f'/content/drive/MyDrive/models/gan/generator.model2',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
  )
  tf.keras.models.save_model(
    discriminator,
    f'/content/drive/MyDrive/models/gan/discriminator.model2',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
  )



In [ ]:
def load_models(epoch):
  generator = tf.keras.models.load_model(f'/content/drive/MyDrive/models/gan/generator.model2')
  discriminator = tf.keras.models.load_model(f'/content/drive/MyDrive/models/gan/discriminator.model2')
  return generator, discriminator

In [ ]:
generator, discriminator = load_models(0)

In [ ]:
EPOCHS = 20000
noise_dim = 100
num_examples_to_generate = 16
BATCH_SIZE = 128

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)



    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
import time

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  #fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      #plt.subplot(4, 4, i+1)
      curr_pred = predictions[i]
      curr_pred = curr_pred.numpy()
      curr_pred = curr_pred * 127.5 + 127.5

      #plt.imshow(curr_pred * 127.5 + 127.5)
      #plt.axis('off')
      filename = f'/content/drive/MyDrive/models/output_images/image{i}_at_epoch_{epoch}.png'
      #print(curr_pred)
      #print(filename)
      if i in [1, 2]:
        cv2_imshow(curr_pred)

  cv2.imwrite(filename, curr_pred)

  #plt.savefig('/content/drive/MyDrive/models/output_images/image_at_epoch_{:04d}.png'.format(epoch))
  #plt.show()

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch, labels in dataset:
      #print(image_batch.shape)
      train_step(image_batch)

    # Produce images for the GIF as you go
    #display.clear_output(wait=True)

    generate_and_save_images(generator,
                             epoch,
                             seed)


    # Save the model every 15 epochs
    if (epoch + 1) % 5 == 0:
      save_models(generator, discriminator, epoch)

    print ('Time for epoch {} is {} sec'.format(epoch, time.time()-start))

  # Generate after the final epoch
  #display.clear_output(wait=True)

  generate_and_save_images(generator,
                           epochs,
                           seed)


In [ ]:
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, list_IDs, labels, batch_size=16, dim=(256,256), n_channels=3,
              n_classes=2, shuffle=True, augmentation=True, segmentation=True):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.labels = labels
    self.list_IDs = list_IDs
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.shuffle = shuffle
    self.augmentation = augmentation
    self.segmentation = segmentation

    if self.augmentation:
      self.seq = iaa.Sequential([
                              iaa.GaussianBlur(sigma=(0.1, 3.5)),
                              iaa.Emboss(alpha=(0.0, 1.0), strength=(0.0, 1.5)),
                              iaa.Fliplr(0.25),
                              iaa.Flipud(0.25),
                              iaa.Affine(rotate=(-45, 45)),
                              iaa.PiecewiseAffine(scale=(0.01, 0.05)),
                              iaa.Affine(shear=(-7, 7))
                          ])

    self.on_epoch_end()

  def __len__(self):
    'Denotes the number of steps per epoch'
    return int(np.floor(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in indexes]
    list_labels_temp = [self.labels[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp, list_labels_temp)

    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
        np.random.shuffle(self.indexes)

  def hair_removal(self, src):
    # Convert the original image to grayscale
    #self.grayScale = cv2.cvtColor( src, cv2.COLOR_RGB2GRAY )
    # Kernel for the morphological filtering
    kernel = cv2.getStructuringElement(1,(17,17))
    # Perform the blackHat filtering on the grayscale image to find the
    # hair countours
    blackhat = cv2.morphologyEx(self.grayScale, cv2.MORPH_BLACKHAT, kernel)
    # intensify the hair countours in preparation for the inpainting
    # algorithm
    ret,thresh2 = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    # inpaint the original image depending on the mask
    dst = cv2.inpaint(src,thresh2,1,cv2.INPAINT_TELEA)
    return dst

  def segment_image(self, image):
    #self.grayScale = cv2.cvtColor( image, cv2.COLOR_RGB2GRAY )
    th = threshold_otsu(self.grayScale)
    mask  = self.grayScale < th
    mask = np.stack((mask,)*3, axis=-1)
    filtered = image * mask
    return filtered


  def augment_image(self, image):
    aug_image = self.seq.augment_image(image)
    return aug_image


  def __data_generation(self, list_IDs_temp, list_labels_temp):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
        # Store sample
        #img = cv2.imread(ID,cv2.IMREAD_GRAYSCALE)
        img = cv2.imread(ID)
        #print(ID)
        img_resized = cv2.resize(img, self.dim[::-1])
        img_resized = cv2.medianBlur(img_resized, 3)
        self.grayScale = cv2.cvtColor( img_resized, cv2.COLOR_RGB2GRAY )
        img_resized = self.hair_removal(img_resized)
        # Segmentation block
        if self.segmentation:
          img_resized = self.segment_image(img_resized)
        # Classical augmentation
        if self.augmentation:
          img_resized = self.augment_image(img_resized)
        # GAN augmentation


        #img_resized = cv2.cvtColor( img_resized, cv2.COLOR_RGB2GRAY )
        #img_resized = np.expand_dims(img_resized, axis=-1)
        X[i, ] = img_resized
        # Store class
        y[i] = list_labels_temp[i]

    return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


In [ ]:
df_malg = df[df['target'] == 1]

In [ ]:
df_malg

In [ ]:
#image = cv2.imread(df_malg['image_name_'].to_list()[7])
#cv2_imshow(image)

In [ ]:
train_generator = DataGenerator(list_IDs= df_malg['image_name_'].to_list(),
                                labels= df_malg['target'].to_list(),
                                n_channels=3,
                                shuffle=True,
                                augmentation=True,
                                batch_size=128,
                                segmentation=False
                                )

In [ ]:
for images, labels in train_generator:
  for image in images:
    cv2_imshow(image)
  break

In [ ]:
EPOCHS

In [ ]:
train(train_generator, EPOCHS)

In [ ]:
noise = tf.random.normal([20, 100])
noise

In [ ]:
#noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

In [ ]:
generated_image = generated_image.numpy()


In [ ]:
generated_image_ = generated_image[0]
generated_image_ = generated_image_ * 127.5 + 127.5

In [ ]:
cv2_imshow(generated_image_)

In [ ]:
generated_image_ = generated_image[1]
generated_image_ = generated_image_ * 127.5 + 127.5

In [ ]:
cv2_imshow(generated_image_)